In [2]:
import lightgbm as lgb
import pandas as pd
import numpy as np
from hyperopt import fmin, tpe, hp
import ipynb.fs.full.utils as utils
from ipynb.fs.full.features import features_independientes_precio, features_dependientes_precio, features_de_csvs
from ipynb.fs.full.features import tipo_de_propiedad_one_hot_encoding

df_train = pd.read_csv('./data/train.csv')
# Para usarse con el submit a Kaggle
df_eval = pd.read_csv('./data/test.csv')

df_train, df_eval = features_de_csvs(df_train, df_eval)

# Randoms solo para asegurarse que los features esten bien hechos
# df_train = df_train.sample(frac=1).reset_index(drop=True)
# df_train = utils.dolarizar_df(df_train)
# df_train = df_train.sample(frac=1).reset_index(drop=True)

df_train, df_test = utils.dividir_df_testeo(df_train, test_size=0.15)

# df_train = df_train.sample(frac=1).reset_index(drop=True)
# df_test = utils.pesificar_df(df_test)
# df_train = df_train.sample(frac=1).reset_index(drop=True)


In [3]:
features_base = ['antiguedad', 'habitaciones', 'garages', 'banos',
       'metroscubiertos', 'metrostotales', 
#                  'lat', 'lng',
       'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas', 'centroscomercialescercanos']

# df_train = df_train.sample(frac=1).reset_index(drop=True)

df_test_f = features_independientes_precio(df_test)
df_test_f = features_dependientes_precio(df_test_f, df_train)
df_test_f, test_nuevas_features_tipo_propiedad = tipo_de_propiedad_one_hot_encoding(df_test_f)

df_train_f = features_independientes_precio(df_train)
df_train_f = features_dependientes_precio(df_train_f, df_train)
df_train_f, train_nuevas_features_tipo_propiedad = tipo_de_propiedad_one_hot_encoding(df_train_f)

df_train_f['fecha'] = pd.to_datetime(df_train_f['fecha']).astype(int)
df_test_f['fecha'] = pd.to_datetime(df_test_f['fecha']).astype(int)

# df_train_f = df_train_f.sample(frac=1).reset_index(drop=True)

## Búsqueda hiperparámetros

In [4]:
def asd(val):
    if val != 1:
        print('--')
        print(val)
    return val

df_train_f['id'].value_counts().apply(asd)

262143    1
78363     1
78375     1
78374     1
78373     1
         ..
170199    1
170197    1
170196    1
170193    1
287281    1
Name: id, Length: 179560, dtype: int64

In [9]:
features = ['antiguedad', 'habitaciones', 'garages', 'banos', 'metroscubiertos', 'metrostotales', 
       'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas', 'centroscomercialescercanos']

features_test = ['prop_frecuente', 'top_provincia', 'porcentaje_metros', 'diferencia_metros', 'promedio_precio_ciudad', 
                 'promedio_por_mes', 'anio', 'promedio_id_zona', 'promedio_precio_tipo_propiedad', 
                 'promedio_precio_habitaciones', 'count_id_zona', 'count_ciudad', 'puntaje', 
                 'count_tipo_propiedad', 'count_tipo_propiedad_ciudad']

features += features_test
features += train_nuevas_features_tipo_propiedad

df_train_f['antiguedad'] = df_train_f['antiguedad'].astype(float)
df_train_f['habitaciones'] = df_train_f['habitaciones'].astype(float)
df_train_f['garages'] = df_train_f['garages'].astype(float)
df_train_f['banos'] = df_train_f['banos'].astype(float)
df_train_f['metroscubiertos'] = df_train_f['metroscubiertos'].astype(float)
df_train_f['metrostotales'] = df_train_f['metrostotales'].astype(float)
df_train_f['gimnasio'] = df_train_f['gimnasio'].astype(float)
df_train_f['usosmultiples'] = df_train_f['usosmultiples'].astype(float)
df_train_f['piscina'] = df_train_f['piscina'].astype(float)
df_train_f['escuelascercanas'] = df_train_f['escuelascercanas'].astype(float)
df_train_f['centroscomercialescercanos'] = df_train_f['centroscomercialescercanos'].astype(float)
df_train_f['prop_frecuente'] = df_train_f['prop_frecuente'].astype(bool)
df_train_f['top_provincia'] = df_train_f['top_provincia'].astype(int)
df_train_f['porcentaje_metros'] = df_train_f['porcentaje_metros'].astype(float)
df_train_f['diferencia_metros'] = df_train_f['diferencia_metros'].astype(float)
df_train_f['promedio_precio_ciudad'] = df_train_f['promedio_precio_ciudad'].astype(float)
df_train_f['promedio_por_mes'] = df_train_f['promedio_por_mes'].astype(float)
df_train_f['anio'] = df_train_f['anio'].astype(int)
df_train_f['promedio_id_zona'] = df_train_f['promedio_id_zona'].astype(float)
df_train_f['promedio_precio_tipo_propiedad'] = df_train_f['promedio_precio_tipo_propiedad'].astype(float)
df_train_f['promedio_precio_habitaciones'] = df_train_f['promedio_precio_habitaciones'].astype(float)
df_train_f['count_id_zona'] = df_train_f['count_id_zona'].astype(float)
df_train_f['count_ciudad'] = df_train_f['count_ciudad'].astype(float)
df_train_f['puntaje'] = df_train_f['puntaje'].astype(int)
df_train_f['count_tipo_propiedad'] = df_train_f['count_tipo_propiedad'].astype(int)
df_train_f['count_tipo_propiedad_ciudad'] = df_train_f['count_tipo_propiedad_ciudad'].astype(float)

df_train_f['tipodepropiedad_Apartamento'] = df_train_f['tipodepropiedad_Apartamento'].astype(int)
df_train_f['tipodepropiedad_Bodega_comercial'] = df_train_f['tipodepropiedad_Bodega_comercial'].astype(int)
df_train_f['tipodepropiedad_Casa'] = df_train_f['tipodepropiedad_Casa'].astype(int)
df_train_f['tipodepropiedad_Casa_en_condominio'] = df_train_f['tipodepropiedad_Casa_en_condominio'].astype(int)
df_train_f['tipodepropiedad_Casa_uso_de_suelo'] = df_train_f['tipodepropiedad_Casa_uso_de_suelo'].astype(int)
df_train_f['tipodepropiedad_Duplex'] = df_train_f['tipodepropiedad_Duplex'].astype(int)
df_train_f['tipodepropiedad_Edificio'] = df_train_f['tipodepropiedad_Edificio'].astype(int)
df_train_f['tipodepropiedad_Inmuebles_productivos_urbanos'] = df_train_f['tipodepropiedad_Inmuebles_productivos_urbanos'].astype(int)
df_train_f['tipodepropiedad_Local_Comercial'] = df_train_f['tipodepropiedad_Local_Comercial'].astype(int)
df_train_f['tipodepropiedad_Oficina_comercial'] = df_train_f['tipodepropiedad_Oficina_comercial'].astype(int)
df_train_f['tipodepropiedad_Quinta_Vacacional'] = df_train_f['tipodepropiedad_Quinta_Vacacional'].astype(int)
df_train_f['tipodepropiedad_Terreno'] = df_train_f['tipodepropiedad_Terreno'].astype(int)
df_train_f['tipodepropiedad_Terreno_comercial'] = df_train_f['tipodepropiedad_Terreno_comercial'].astype(int)
df_train_f['tipodepropiedad_Villa'] = df_train_f['tipodepropiedad_Villa'].astype(int)
df_train_f['tipodepropiedad_otro'] = df_train_f['tipodepropiedad_otro'].astype(int)
df_train_f['tipodepropiedad_Otros'] = df_train_f['tipodepropiedad_Otros'].astype(int)


df_train_f[features].dtypes

antiguedad                                       float64
habitaciones                                     float64
garages                                          float64
banos                                            float64
metroscubiertos                                  float64
metrostotales                                    float64
gimnasio                                         float64
usosmultiples                                    float64
piscina                                          float64
escuelascercanas                                 float64
centroscomercialescercanos                       float64
prop_frecuente                                      bool
top_provincia                                      int64
porcentaje_metros                                float64
diferencia_metros                                float64
promedio_precio_ciudad                           float64
promedio_por_mes                                 float64
anio                           

In [10]:
features = ['antiguedad', 'habitaciones', 'garages', 'banos', 'metroscubiertos', 'metrostotales', 
       'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas', 'centroscomercialescercanos']

features_test = ['prop_frecuente', 'top_provincia', 'porcentaje_metros', 'diferencia_metros', 'promedio_precio_ciudad', 
                 'promedio_por_mes', 'anio', 'promedio_id_zona', 'promedio_precio_tipo_propiedad', 
                 'promedio_precio_habitaciones', 'count_id_zona', 'count_ciudad', 'puntaje', 
                 'count_tipo_propiedad', 'count_tipo_propiedad_ciudad']

features += features_test
features += train_nuevas_features_tipo_propiedad

df_test_f['antiguedad'] = df_test_f['antiguedad'].astype(float)
df_test_f['habitaciones'] = df_test_f['habitaciones'].astype(float)
df_test_f['garages'] = df_test_f['garages'].astype(float)
df_test_f['banos'] = df_test_f['banos'].astype(float)
df_test_f['metroscubiertos'] = df_test_f['metroscubiertos'].astype(float)
df_test_f['metrostotales'] = df_test_f['metrostotales'].astype(float)
df_test_f['gimnasio'] = df_test_f['gimnasio'].astype(float)
df_test_f['usosmultiples'] = df_test_f['usosmultiples'].astype(float)
df_test_f['piscina'] = df_test_f['piscina'].astype(float)
df_test_f['escuelascercanas'] = df_test_f['escuelascercanas'].astype(float)
df_test_f['centroscomercialescercanos'] = df_test_f['centroscomercialescercanos'].astype(float)
df_test_f['prop_frecuente'] = df_test_f['prop_frecuente'].astype(bool)
df_test_f['top_provincia'] = df_test_f['top_provincia'].astype(int)
df_test_f['porcentaje_metros'] = df_test_f['porcentaje_metros'].astype(float)
df_test_f['diferencia_metros'] = df_test_f['diferencia_metros'].astype(float)
df_test_f['promedio_precio_ciudad'] = df_test_f['promedio_precio_ciudad'].astype(float)
df_test_f['promedio_por_mes'] = df_test_f['promedio_por_mes'].astype(float)
df_test_f['anio'] = df_test_f['anio'].astype(int)
df_test_f['promedio_id_zona'] = df_test_f['promedio_id_zona'].astype(float)
df_test_f['promedio_precio_tipo_propiedad'] = df_test_f['promedio_precio_tipo_propiedad'].astype(float)
df_test_f['promedio_precio_habitaciones'] = df_test_f['promedio_precio_habitaciones'].astype(float)
df_test_f['count_id_zona'] = df_test_f['count_id_zona'].astype(float)
df_test_f['count_ciudad'] = df_test_f['count_ciudad'].astype(float)
df_test_f['puntaje'] = df_test_f['puntaje'].astype(int)
df_test_f['count_tipo_propiedad'] = df_test_f['count_tipo_propiedad'].astype(int)
df_test_f['count_tipo_propiedad_ciudad'] = df_test_f['count_tipo_propiedad_ciudad'].astype(float)

df_test_f['tipodepropiedad_Apartamento'] = df_test_f['tipodepropiedad_Apartamento'].astype(int)
df_test_f['tipodepropiedad_Bodega_comercial'] = df_test_f['tipodepropiedad_Bodega_comercial'].astype(int)
df_test_f['tipodepropiedad_Casa'] = df_test_f['tipodepropiedad_Casa'].astype(int)
df_test_f['tipodepropiedad_Casa_en_condominio'] = df_test_f['tipodepropiedad_Casa_en_condominio'].astype(int)
df_test_f['tipodepropiedad_Casa_uso_de_suelo'] = df_test_f['tipodepropiedad_Casa_uso_de_suelo'].astype(int)
df_test_f['tipodepropiedad_Duplex'] = df_test_f['tipodepropiedad_Duplex'].astype(int)
df_test_f['tipodepropiedad_Edificio'] = df_test_f['tipodepropiedad_Edificio'].astype(int)
df_test_f['tipodepropiedad_Inmuebles_productivos_urbanos'] = df_test_f['tipodepropiedad_Inmuebles_productivos_urbanos'].astype(int)
df_test_f['tipodepropiedad_Local_Comercial'] = df_test_f['tipodepropiedad_Local_Comercial'].astype(int)
df_test_f['tipodepropiedad_Oficina_comercial'] = df_test_f['tipodepropiedad_Oficina_comercial'].astype(int)
df_test_f['tipodepropiedad_Quinta_Vacacional'] = df_test_f['tipodepropiedad_Quinta_Vacacional'].astype(int)
df_test_f['tipodepropiedad_Terreno'] = df_test_f['tipodepropiedad_Terreno'].astype(int)
df_test_f['tipodepropiedad_Terreno_comercial'] = df_test_f['tipodepropiedad_Terreno_comercial'].astype(int)
df_test_f['tipodepropiedad_Villa'] = df_test_f['tipodepropiedad_Villa'].astype(int)
df_test_f['tipodepropiedad_otro'] = df_test_f['tipodepropiedad_otro'].astype(int)
df_test_f['tipodepropiedad_Otros'] = df_test_f['tipodepropiedad_Otros'].astype(int)


df_test_f[features].dtypes

antiguedad                                       float64
habitaciones                                     float64
garages                                          float64
banos                                            float64
metroscubiertos                                  float64
metrostotales                                    float64
gimnasio                                         float64
usosmultiples                                    float64
piscina                                          float64
escuelascercanas                                 float64
centroscomercialescercanos                       float64
prop_frecuente                                      bool
top_provincia                                      int64
porcentaje_metros                                float64
diferencia_metros                                float64
promedio_precio_ciudad                           float64
promedio_por_mes                                 float64
anio                           

In [12]:
features = ['antiguedad', 'habitaciones', 'garages', 'banos',
       'metroscubiertos', 'metrostotales', 
#             'lat', 'lng',
       'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas', 'centroscomercialescercanos']

features_test = ['prop_frecuente', 'top_provincia', 'porcentaje_metros', 'diferencia_metros', 'promedio_precio_ciudad', 
                 'promedio_por_mes', 'anio', 'promedio_id_zona', 'promedio_precio_tipo_propiedad', 
                 'promedio_precio_habitaciones', 'count_id_zona', 'count_ciudad', 'puntaje', 
                 'count_tipo_propiedad', 'count_tipo_propiedad_ciudad']

features += features_test
features += train_nuevas_features_tipo_propiedad

display()


def eval_lightgbm(args):
    num_leaves, learning_rate, feature_fraction, bagging_fraction, bagging_freq, max_depth, test_size = args
    
    df_train_h = df_train_f.sample(frac=1).reset_index(drop=True)
    
    x_train, x_test, y_train, y_test = utils.dividir_dataset(df_train_h, 'precio', features, test_size=test_size)

    lgb_train = lgb.Dataset(x_train, y_train)
    lgb_eval = lgb.Dataset(x_test, y_test, reference=lgb_train)
    
    num_leaves = int(num_leaves)
    bagging_freq = int(bagging_freq)
    max_depth = int(max_depth)
    params = {
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'metric': {'mae'}, # Si se deja vacio se toma el ideal para llegar al 'objective'
        'num_leaves': num_leaves,
        'learning_rate': learning_rate,
        'feature_fraction': feature_fraction,
        'bagging_fraction': bagging_fraction,
        'bagging_freq': bagging_freq,
        'max_depth': max_depth,
        'verbose': -1,
    }

    gbm = lgb.train(params,
                    lgb_train,
                    valid_sets=lgb_eval,
                    num_boost_round=150,
                    early_stopping_rounds=15,
                    verbose_eval=-1)
    
    y_pred_test = gbm.predict(utils.filtrar_features(df_test_f, features), num_iteration=gbm.best_iteration)
    return utils.MAE(df_test_f['precio'].values, y_pred_test)

space = [hp.quniform('num_leaves', 30, 130, 1), hp.uniform('learning_rate', 0.05, 0.9),
        hp.uniform('feature_fraction', 0.10, 0.90), hp.uniform('bagging_fraction', 0.10, 0.90),
        hp.quniform('bagging_freq', 1, 130, 1), hp.quniform('max_depth', 1, 20, 1),
        hp.uniform('test_size', 0.01, 0.4)]

hps = fmin(eval_lightgbm, space=space, algo=tpe.suggest, max_evals=100, verbose=1)

display(hps)

  0%|          | 0/100 [00:00<?, ?it/s, best loss: ?]

id                                   object
titulo                               object
descripcion                          object
tipodepropiedad                      object
direccion                            object
                                      ...  
tipodepropiedad_Quinta_Vacacional     int64
tipodepropiedad_Terreno               int64
tipodepropiedad_Terreno_comercial     int64
tipodepropiedad_Villa                 int64
tipodepropiedad_otro                  int64
Length: 89, dtype: object

  0%|          | 0/100 [00:01<?, ?it/s, best loss: ?]


ValueError: Series.dtypes must be int, float or bool

## Evaluación features

In [ ]:
features = ['antiguedad', 'habitaciones', 'garages', 'banos',
       'metroscubiertos', 'metrostotales', 
#             'lat', 'lng',
       'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas', 'centroscomercialescercanos']

features_test = ['promedio_precio_hbg_tipo_propiedad', 'promedio_precio_hbg_tipo_propiedad_provincia',
                'promedio_precio_hbg_tipo_propiedad_provincia_gen']

hps = {'bagging_fraction': 0.806451877022587,
 'bagging_freq': 62.0,
 'feature_fraction': 0.5379925983440028,
 'learning_rate': 0.1363027714646826,
 'max_depth': 11.0,
 'num_leaves': 113.0,
 'test_size': 0.09575190901892519}

# hps = {'bagging_fraction': 0.7514713723096431,
#  'bagging_freq': 97.0,
#  'feature_fraction': 0.7403311365563734,
#  'learning_rate': 0.11573281219786402,
#  'max_depth': 13.0,
#  'num_leaves': 87.0,
#  'test_size': 0.12116948662473831}


bagging_fraction = hps['bagging_fraction']
bagging_freq = int(hps['bagging_freq'])
feature_fraction = hps['feature_fraction']
learning_rate = hps['learning_rate']
num_leaves = int(hps['num_leaves'])
max_depth = int(hps['max_depth'])
test_size = hps['test_size']

params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'mae'}, # Si se deja vacio se toma el ideal para llegar al 'objective'
    'num_leaves': num_leaves,
    'learning_rate': learning_rate,
    'feature_fraction': feature_fraction,
    'bagging_fraction': bagging_fraction,
    'bagging_freq': bagging_freq,
    'max_depth': max_depth,
    'verbose': 0
}


base_train = 0
base_test = 0
base_eval = 0
for i in ['None'] + features_test:
    x_train, x_test, y_train, y_test = utils.dividir_dataset(df_train_f, 'precio', features + [i], test_size=test_size)

    lgb_train = lgb.Dataset(x_train, y_train)
    lgb_eval = lgb.Dataset(x_test, y_test, reference=lgb_train)

    gbm = lgb.train(params,
                    lgb_train,
                    valid_sets=lgb_eval,
                    num_boost_round=1000,
                    early_stopping_rounds=15,
                    verbose_eval=-1)

    y_pred_test = gbm.predict(x_test, raw_score=True, num_iteration=gbm.best_iteration)
    y_pred_train = gbm.predict(x_train, raw_score=True, num_iteration=gbm.best_iteration)
    y_pred_eval = gbm.predict(utils.filtrar_features(df_test_f, features + [i]), num_iteration=gbm.best_iteration)

    gbm_mae_train = utils.MAE(y_train, y_pred_train)
    gbm_mae = utils.MAE(y_test, y_pred_test)
    gbm_mae_eval = utils.MAE(df_test_f['precio'].values, y_pred_eval)

    print(f"MAE LightGBM (train): {gbm_mae_train:.5f}")
    print(f"MAE LightGBM (test): {gbm_mae:.5f}")
    print(f"MAE LightGBM (eval): {gbm_mae_eval:.5f}")
    if i is not 'None':
        print(f"Overfitting (base_eval - base_test) - (eval - test) - {i}: {(base_eval - base_test) - (gbm_mae_eval - gbm_mae)}")
        print(f"Diff evaluation (base_eval - eval)                  - {i}: {base_eval - gbm_mae_eval}")
        print(f"Diff train (base_train - train)                     - {i}: {base_train - gbm_mae_train}")
    else:
        base_train = gbm_mae_train
        base_test = gbm_mae
        base_eval = gbm_mae_eval

In [ ]:
df_train_f.columns

## Evaluación modelo final

In [ ]:
features = ['habitaciones', 'garages', 'banos',
       'metroscubiertos', 'metrostotales', 
            'lat', 'lng',
       'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas', 'centroscomercialescercanos',
           'fecha']


features_test = ['prop_frecuente', 'top_provincia', 'porcentaje_metros', 'diferencia_metros', 'promedio_precio_ciudad', 
                 'promedio_por_mes', 'anio', 'promedio_id_zona', 'promedio_precio_tipo_propiedad', 
                 'promedio_precio_hbg_tipo_propiedad', 'count_id_zona', 'count_ciudad', 'puntaje', 
                 'count_tipo_propiedad', 'count_tipo_propiedad_ciudad', 'promedio_id_zona_gen', 
                 'promedio_precio_tipo_propiedad_ciudad_gen', 'promedio_precio_hbg_tipo_propiedad_provincia']

# features_test = ['promedio_precio_ciudad',
#                 'promedio_id_zona']

features += features_test


hps = {'bagging_fraction': 0.806451877022587,
 'bagging_freq': 62.0,
 'feature_fraction': 0.5379925983440028,
 'learning_rate': 0.1363027714646826,
 'max_depth': 11.0,
 'num_leaves': 113.0,
 'test_size': 0.09575190901892519}


bagging_fraction = hps['bagging_fraction']
bagging_freq = int(hps['bagging_freq'])
feature_fraction = hps['feature_fraction']
learning_rate = hps['learning_rate']
num_leaves = int(hps['num_leaves'])
max_depth = int(hps['max_depth'])
test_size = hps['test_size']

params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'mae'}, # Si se deja vacio se toma el ideal para llegar al 'objective'
    'num_leaves': num_leaves,
    'learning_rate': learning_rate,
    'feature_fraction': feature_fraction,
    'bagging_fraction': bagging_fraction,
    'bagging_freq': bagging_freq,
    'max_depth': max_depth,
    'verbose': 0
}

x_train, x_test, y_train, y_test = utils.dividir_dataset(df_train_f, 'precio', features, test_size=test_size)

lgb_train = lgb.Dataset(x_train, y_train)
lgb_eval = lgb.Dataset(x_test, y_test, reference=lgb_train)

gbm = lgb.train(params,
                lgb_train,
                valid_sets=lgb_eval,
                num_boost_round=1000,
                early_stopping_rounds=25,
                verbose_eval=1)

y_pred_test = gbm.predict(x_test, num_iteration=gbm.best_iteration)
y_pred_train = gbm.predict(x_train, num_iteration=gbm.best_iteration)
y_pred_eval = gbm.predict(utils.filtrar_features(df_test_f, features), num_iteration=gbm.best_iteration)

df_test_f['target'] = y_pred_eval
# df_test_f = utils.pesificar_df(df_test_f, 'target', 'target')

gbm_mae_train = utils.MAE(y_train, y_pred_train)
gbm_mae = utils.MAE(y_test, y_pred_test)
gbm_mae_eval = utils.MAE(df_test_f['precio'].values, df_test_f['target'].values)

print(f"MAE LightGBM (train): {gbm_mae_train:.5f}")
print(f"MAE LightGBM (test): {gbm_mae:.5f}")
print(f"MAE LightGBM (eval): {gbm_mae_eval:.5f}")

In [ ]:
import shap 
shap.initjs()

df_test_f['diff'] = df_test_f['precio'].values - df_test_f['target'].values
df_test_f_frac = df_test_f.sort_values('diff', ascending=False).iloc[0:1000]
df_test_f_frac = utils.filtrar_features(df_test_f_frac, features)

explainer = shap.TreeExplainer(gbm)
shap_values = explainer.shap_values(df_test_f_frac)

In [ ]:
shap.force_plot(explainer.expected_value, shap_values[1,:], df_test_frac.iloc[1,:], )

In [ ]:
shap.force_plot(explainer.expected_value, shap_values, df_test_f_frac)

In [ ]:
shap.summary_plot(shap_values, df_test_f_frac)

In [ ]:
shap.dependence_plot("puntaje", shap_values, df_test_f_frac)

## Evaluación df_test completo

In [ ]:
df_train = pd.read_csv('./data/train.csv')

# Para usarse con el submit a Kaggle
df_test = pd.read_csv('./data/test.csv')

df_train, df_test = features_de_csvs(df_train, df_test)

# df_train = utils.dolarizar_df(df_train)

df_test_f = features_independientes_precio(df_test)
df_test_f = features_dependientes_precio(df_test_f, df_train)

df_train_f = features_independientes_precio(df_train)
df_train_f = features_dependientes_precio(df_train_f, df_train)

features = ['antiguedad', 'habitaciones', 'garages', 'banos',
       'metroscubiertos', 'metrostotales', 'lat', 'lng',
       'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas', 'centroscomercialescercanos']

features_test = ['prop_frecuente', 'top_provincia', 'porcentaje_metros', 'diferencia_metros', 'promedio_precio_ciudad', 
                 'promedio_por_mes', 'anio', 'promedio_id_zona', 'promedio_precio_tipo_propiedad', 
                 'promedio_precio_hbg_tipo_propiedad', 'count_id_zona', 'count_ciudad', 'puntaje', 
                 'count_tipo_propiedad', 'count_tipo_propiedad_ciudad', 'promedio_id_zona_gen', 
                 'promedio_precio_tipo_propiedad_ciudad_gen']

features += features_test


hps = {'bagging_fraction': 0.806451877022587,
 'bagging_freq': 62.0,
 'feature_fraction': 0.5379925983440028,
 'learning_rate': 0.1363027714646826,
 'max_depth': 11.0,
 'num_leaves': 113.0,
 'test_size': 0.09575190901892519}

bagging_fraction = hps['bagging_fraction']
bagging_freq = int(hps['bagging_freq'])
feature_fraction = hps['feature_fraction']
learning_rate = hps['learning_rate']
num_leaves = int(hps['num_leaves'])
max_depth = int(hps['max_depth'])
test_size = hps['test_size']

params = {
    'boosting_type': 'dart',
    'objective': 'regression',
    'metric': {'mae'}, # Si se deja vacio se toma el ideal para llegar al 'objective'
    'num_leaves': num_leaves,
    'learning_rate': learning_rate,
    'feature_fraction': feature_fraction,
    'bagging_fraction': bagging_fraction,
    'bagging_freq': bagging_freq,
    'max_depth': max_depth,
    'verbose': 0
}

x_train, x_test, y_train, y_test = utils.dividir_dataset(df_train_f, 'precio', features, test_size=test_size)

lgb_train = lgb.Dataset(x_train, y_train)
lgb_eval = lgb.Dataset(x_test, y_test, reference=lgb_train)


gbm = lgb.train(params,
                lgb_train,
                valid_sets=lgb_eval,
                num_boost_round=1800,
                early_stopping_rounds=15,
                verbose_eval=1)

y_pred_test = gbm.predict(x_test, raw_score=True, num_iteration=gbm.best_iteration)
y_pred_train = gbm.predict(x_train, raw_score=True, num_iteration=gbm.best_iteration)
y_pred_eval = gbm.predict(utils.filtrar_features(df_test_f, features), num_iteration=gbm.best_iteration)

gbm_mae_train = utils.MAE(y_train, y_pred_train)
gbm_mae = utils.MAE(y_test, y_pred_test)

print(f"MAE LightGBM (train): {gbm_mae_train:.5f}")
print(f"MAE LightGBM (test): {gbm_mae:.5f}")

In [ ]:
y_pred_eval = gbm.predict(utils.filtrar_features(df_test_f, features), num_iteration=gbm.best_iteration)
df_test_f['target'] = y_pred_eval

# df_test_f = utils.pesificar_df(df_test_f, 'target', 'target')
df_test_f[['id', 'target']].to_csv('respuesta14.csv', index=False)

In [ ]:
x_test.shape

In [ ]:
df_test_f.shape

In [ ]:
utils.filtrar_features(df_test_f, features).columns